In [2]:
import numpy as np
import pandas as pd
from sunpy.net import hek
from shapely import wkt
from sunpy.time import parse_time
import datetime

In [3]:
client = hek.HEKClient()
start_time = pd.to_datetime('2014/01/01 00:00')
end_time = pd.to_datetime('2014/01/10 00:00')

       

In [258]:
fl_SSW = client.query(hek.attrs.Time(start_time.isoformat(),end_time.isoformat()),
                                  hek.attrs.EventType('FL'), hek.attrs.FRM.Name == 'SSW Latest Events')
fl_SSW = pd.DataFrame(fl_SSW)

In [11]:
ef_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('EF'))
ef = pd.DataFrame(ef_events)

In [4]:
ar_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('AR'))
ar = pd.DataFrame(ar_events)

In [5]:
cme_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('CE'))
cme = pd.DataFrame(cme_events)

In [6]:
sg_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('SG'))
sg = pd.DataFrame(sg_events)

In [14]:
fe_events = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('FE'))
fe = pd.DataFrame(fe_events)

In [5]:
# fl_detective = client.query(hek.attrs.Time(start_time.isoformat(),
#                                                  end_time.isoformat()),
#                                   hek.attrs.EventType('FL'),
#                                   hek.attrs.FRM.Name == 'Flare Detective - Trigger Module')
# fl_SSW= client.query(hek.attrs.Time(start_time.isoformat(),
#                                                  end_time.isoformat()),
#                                   hek.attrs.EventType('FL'),
#                                   hek.attrs.FRM.Name == 'SSW Latest Events')

In [242]:
#may at some point add location of other flares in as well as peak flux
def determine_wavelengths(index, index2 = None):
    if index2 == None:
        if fl_detective['obs_channelid'].values[index] == '131':
            fl_detective['is_131'].values[index] = 1
            fl_detective['fl_peakflux_131'].values[index] = fl_detective['fl_peakflux'].values[index]
            
        if fl_detective['obs_channelid'].values[index] == '171':
            fl_detective['is_171'].values[index] = 1
            fl_detective['fl_peakflux_171'].values[index] = fl_detective['fl_peakflux'].values[index]
        
        if fl_detective['obs_channelid'].values[index] == '193':
            fl_detective['is_193'].values[index] = 1
            fl_detective['fl_peakflux_193'].values[index] = fl_detective['fl_peakflux'].values[index]
        
        if fl_detective['obs_channelid'].values[index] == '211':
            fl_detective['is_211'].values[index] = 1
            fl_detective['fl_peakflux_211'].values[index] = fl_detective['fl_peakflux'].values[index]
        
        if fl_detective['obs_channelid'].values[index] == '304':
            fl_detective['is_304'].values[index] = 1
            fl_detective['fl_peakflux_304'].values[index] = fl_detective['fl_peakflux'].values[index]
        
        if fl_detective['obs_channelid'].values[index] == '355':
            fl_detective['is_355'].values[index] = 1
            fl_detective['fl_peakflux_355'].values[index] = fl_detective['fl_peakflux'].values[index]
        
        if fl_detective['obs_channelid'].values[index] == '94':
            fl_detective['is_94'].values[index] = 1
            fl_detective['fl_peakflux_94'].values[index] = fl_detective['fl_peakflux'].values[index]
    else:
        if fl_detective['obs_channelid'].values[index2] == '131':
            fl_detective['is_131'].values[index] = 1
            fl_detective['fl_peakflux_131'].values[index] = fl_detective['fl_peakflux'].values[index2]
        
        if fl_detective['obs_channelid'].values[index2] == '171':
            fl_detective['is_171'].values[index] = 1
            fl_detective['fl_peakflux_171'].values[index] = fl_detective['fl_peakflux'].values[index2]
        
        if fl_detective['obs_channelid'].values[index2] == '193':
            fl_detective['is_193'].values[index] = 1
            fl_detective['fl_peakflux_193'].values[index] = fl_detective['fl_peakflux'].values[index2]
        
        if fl_detective['obs_channelid'].values[index2] == '211':
            fl_detective['is_211'].values[index] = 1
            fl_detective['fl_peakflux_211'].values[index] = fl_detective['fl_peakflux'].values[index2]
           
        if fl_detective['obs_channelid'].values[index2] == '304':
            fl_detective['is_304'].values[index] = 1
            fl_detective['fl_peakflux_304'].values[index] = fl_detective['fl_peakflux'].values[index2]
        
        if fl_detective['obs_channelid'].values[index2] == '355':
            fl_detective['is_355'].values[index] = 1
            fl_detective['fl_peakflux_355'].values[index] = fl_detective['fl_peakflux'].values[index2]
        
        if fl_detective['obs_channelid'].values[index2] == '94':
            fl_detective['is_94'].values[index] = 1
            fl_detective['fl_peakflux_94'].values[index] = fl_detective['fl_peakflux'].values[index2]

    return df

In [243]:
def fl_duplicates_detective(fl_detective, delta_t_max, distance_max, search_index):
    delete_index = []
    num_del = 0
    num_duplicates = 0
    i = 0
    associated_fl = []
    detective_start = map(parse_time, fl_detective['event_starttime'])
    detective_position = map(lambda x: wkt.loads(x), fl_detective['hpc_coord'])
    delta_t_max = datetime.timedelta(minutes = delta_t_max)
    num_events = fl_detective.shape[0]
    
    #create columns in dataframe for recording multiple wavelength components of flares
    zero_list = [0 for n in range(num_events)]
    is_associated_fl = zero_list
    
    fl_detective.loc[:, 'is_131'] = zero_list
    fl_detective.loc[:, 'is_171'] = zero_list
    fl_detective.loc[:, 'is_193'] = zero_list
    fl_detective.loc[:, 'is_211'] = zero_list
    fl_detective.loc[:, 'is_304'] = zero_list
    fl_detective.loc[:, 'is_355'] = zero_list
    fl_detective.loc[:, 'is_355'] = zero_list
    fl_detective.loc[:, 'is_94'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_131'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_171'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_193'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_211'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_304'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_355'] = zero_list
    fl_detective.loc[:, 'fl_peakflux_94'] = zero_list
    for elem in detective_start:
        fl_match = []
        determine_wavelengths(fl_detective, i)
        #test whether next entry is redundant
        if i < num_events-1:
            if fl_detective['obs_channelid'].values[i] == fl_detective['obs_channelid'].values[i+1]:
                if detective_start[i+1]-elem <= delta_t_max:
                    if detective_position[i].distance(detective_position[i+1]) <= distance_max:
                        delete_index.append(i+1)
                        fl_match.append(fl_detective['SOL_standard'].values[i+1])
                        num_del+=1
        #test whether there are entries in diff wavelengths for the same flare
        for j in range(1, search_index):
            i2 = i+j
            if i2 < (num_events-1):
                if fl_detective['obs_channelid'].values[i] != fl_detective['obs_channelid'].values[i2]:
                    if detective_start[i2]-elem <= delta_t_max:
                        if detective_position[i].distance(detective_position[i2]) <= distance_max:
                            delete_index.append(i2)
                            num_duplicates+=1
                            fl_match.append(str(fl_detective['SOL_standard'].values[i2]))
                            determine_wavelengths(fl_detective, i, i2)
        if fl_match == []: 
             fl_match = [0]
        associated_fl.append(fl_match)
        i+=1
    fl_detective.loc[:, 'associated_fl'] = associated_fl
    k = 0
    for elem in associated_fl:
        if elem!=[0]:
            is_associated_fl[k] = 1
        k+=1
    delete_index = set(delete_index)
    fl_detective.loc[:, 'is_associated_fl'] = is_associated_fl
    fl_detective = fl_detective.drop(delete_index) 
    
    print 'original number of events:  %d' % num_events
    print 'duplicated events deleted:  %d' % num_del
    print 'duplicated events merged:  %d' % (len(delete_index)-num_del)
    print 'new number of events:  %d' % (num_events-len(delete_index))

    return fl_detective     

In [190]:
fl_detective = pd.DataFrame(fl_detective)
fl_detective_clean = fl_duplicates_detective(fl_detective, 20, 60, 10)
fl_detective_clean = add_dist_frm_center_column(fl_detective_clean)
flare_keywords = list(np.genfromtxt('keywords_flare.csv', delimiter=',', dtype=str))
flare_keywords.extend(['is_associated_fl', 'associated_fl', 'is_131', 'is_171', 'is_193', 'is_211', 'is_304', 
                          'is_355','is_94', 'fl_peakflux_131', 'fl_peakflux_171','fl_peakflux_193', 'fl_peakflux_211',
                          'fl_peakflux_304', 'fl_peakflux_355', 'fl_peakflux_94', 'dist_frm_center'])
fl_detective_clean.to_csv('flares_database.csv', index = False, columns = flare_keywords)

original number of events: 2012
duplicated events deleted: 81
duplicated events merged: 812
new number of events: 1119


In [276]:
delta_t_max = 10 #[minutes]
distance_max = 5 #[arcsec, HPC]
keywords_SSW = ['SOL_standard', 'event_starttime', 'event_endtime', 'event_peaktime', 'hpc_coord', 
                'hpc_bbox', 'fl_goescls']

def fl_duplicates_SSW(fl_SSW, delta_t_max, distance_max):
    delete_index = []
    i = 0
    print(fl_SSW)
    SSW_start = fl_SSW['event_starttime']
    SSW_start = map(parse_time, SSW_start)
    SSW_position = map(lambda x: wkt.loads(x), fl_SSW['hpc_coord'])
    delta_t_max = datetime.timedelta(minutes = delta_t_max)
    
    for elem in SSW_start:
        j = 1
        is_duplicate = True
        while is_duplicate == True and i+j < len(SSW_start):
            is_duplicate = False
            if fl_SSW['fl_goescls'].values[i] == fl_SSW['fl_goescls'].values[i+j]:
                if SSW_start[i+j]-elem <= delta_t_max:
                    if SSW_position[i].distance(SSW_position[i+j]) <= distance_max:
                        delete_index.append(i+j)
                        is_duplicate = True
                        j+=1
        i+=1
    fl_SSW = fl_SSW.drop(delete_index)
    print delete_index
    print fl_SSW.shape
        
    return fl_SSW     

In [277]:
delta_t_max = 10
distance_max = 10
fl_SSW = fl_duplicates_SSW(fl_SSW, delta_t_max, distance_max)

                      SOL_standard active ar_compactnesscls ar_mcintoshcls  \
0   SOL2014-01-01T06:45:00L096C094   true                                    
1   SOL2014-01-01T06:45:00L096C094   true                                    
2   SOL2014-01-01T07:21:00L233C102   true                                    
3   SOL2014-01-01T15:40:00L091C096   true                                    
4   SOL2014-01-01T18:40:00L225C106   true                                    
5   SOL2014-01-01T21:54:00L088C097   true                                    
6   SOL2014-01-01T22:30:00L088C097   true                                    
7   SOL2014-01-02T02:24:00L085C095   true                                    
8   SOL2014-01-02T04:25:00L086C097   true                                    
9   SOL2014-01-02T07:43:00L093C095   true                                    
10  SOL2014-01-02T07:43:00L093C095   true                                    
11  SOL2014-01-02T11:25:00L089C091   true                       

In [ ]:
#match flares from flare_detective with GOES class
#locations/times don't precisely match up between SSW entries and flare_detective entries...even though they are being 
#collected from the same satellite...

def fl_goes_class():

In [134]:
def create_fl_database(start_time, end_time, delta_t_max, distance_max):
    fl_detective = client.query(hek.attrs.Time(start_time.isoformat(),end_time.isoformat()), 
                                hek.attrs.EventType('FL'), hek.attrs.FRM.Name == 'Flare Detective - Trigger Module')
    fl_detective = pd.DataFrame(fl_detective)
#     fl_SSW= client.query(hek.attrs.Time(start_time.isoformat(),end_time.isoformat()), hek.attrs.EventType('FL'), 
#                          hek.attrs.FRM.Name == 'SSW Latest Events')
#     fl_SSW = pd.Dataframe(fl_SSW)
    delta_t_max = datetime.timedelta(minutes = delta_t_max)
    fl_detective = fl_duplicates_detective(delta_t_max, distance_max)
#     fl_SSW = fl_duplicates_SSW(delta_t_max, distance_max)
    
    flare_keywords = list(np.genfromtxt('keywords_flare.csv', delimiter=',', dtype=str))
    flare_keywords.extend(['is_associated_fl', 'associated_fl', 'is_131', 'is_171', 'is_193', 'is_211', 'is_304', 
                          'is_355','is_94', 'fl_peakflux_131', 'fl_peakflux_171','fl_peakflux_193', 'fl_peakflux_211',
                          'fl_peakflux_304', 'fl_peakflux_355', 'fl_peakflux_94', 'fl_goescls'])
    fl_detective.to_csv('flares_database.csv', index = False, columns = flare_keywords)

    
    

In [188]:
def find_dist_frm_center(x, y):
    r = 966
    z_sq = (r**2 - x**2 - y**2)
    if z_sq >= 0: z = z_sq**(0.5)
    else: z = (-z_sq)**(0.5)
    dist = (x**2 + y**2 + (z-r)**2)**(0.5)
    return dist

In [189]:
def add_dist_frm_center_column(df):
    dist_frm_center = []
    for i in range(df.shape[0]):
        dist = find_dist_frm_center(df['hpc_x'].values[i], df['hpc_y'].values[i])
        dist_frm_center.append(dist)
    df.loc[:, 'dist_frm_center'] = dist_frm_center
    return df
    

In [121]:
#TEST

fl_detective = client.query(hek.attrs.Time(start_time.isoformat(),end_time.isoformat()), 
                                hek.attrs.EventType('FL'), hek.attrs.FRM.Name == 'Flare Detective - Trigger Module')

In [7]:
#export from HEK SSW, SWPC, & flare detective events separately
#assume dupilcates have already been merged together
def combine_flares_SSW_SWPC():
    for elem in fl_SSW['event_starttime']
    fl_SWPC
    

3807


In [12]:
flare_keywords = list(np.genfromtxt('flare_keywords.csv', delimiter=',', dtype=str))
flare_keywords.append('is_associated_fl', 'associated_fl', 'is_131', 'is_171', 'is_193', 'is_211', 'is_304', 'is_355',
                     'is_94', 'fl_peakflux_131', 'fl_peakflux_171','fl_peakflux_193', 'fl_peakflux_211',
                     'fl_peakflux_304', 'fl_peakflux_355', 'fl_peakflux_94', 'fl_goescls')
ef_keywords = list(np.genfromtxt('ef_keywords.csv', delimiter=',', dtype =str))
# print flare_keywords
# print ef_keywords
#to change ordering of columns/delete excess ones: df = df[list_keywords]


                       SOL_standard      event_starttime        event_endtime  \
0    SOL2013-12-31T23:30:45L151C090  2013-12-31T23:30:45  2014-01-01T02:18:45   
1    SOL2013-12-31T23:42:45L233C077  2013-12-31T23:42:45  2014-01-01T01:18:45   
2    SOL2014-01-01T00:54:45L223C079  2014-01-01T00:54:45  2014-01-01T06:42:45   
3    SOL2014-01-01T02:18:45L206C084  2014-01-01T02:18:45  2014-01-01T03:42:45   
4    SOL2014-01-01T02:18:45L223C117  2014-01-01T02:18:45  2014-01-01T03:54:45   
5    SOL2014-01-01T02:30:45L233C103  2014-01-01T02:30:45  2014-01-01T04:54:45   
6    SOL2014-01-01T03:42:45L207C084  2014-01-01T03:42:45  2014-01-01T05:42:45   
7    SOL2014-01-01T07:18:45L232C103  2014-01-01T07:18:45  2014-01-01T09:18:45   
8    SOL2014-01-01T07:18:45L211C104  2014-01-01T07:18:45  2014-01-01T12:30:45   
9    SOL2014-01-01T07:54:45L234C094  2014-01-01T07:54:45  2014-01-01T09:30:45   
10   SOL2014-01-01T07:54:45L215C088  2014-01-01T07:54:45  2014-01-01T11:30:45   
11   SOL2014-01-01T08:06:45L

In [15]:
# flare.to_csv('look_at_flare_data.csv')
# ef.to_csv('look_at_ef_data.csv')
# ar.to_csv('look_at_ar_data.csv')
# cme.to_csv('look_at_cme_data.csv')
# sg.to_csv('look_at_sg_data.csv')
# fe.to_csv('look_at_fe_data.csv')

In [ ]:
#final step to export dataframe to csv

# fl.to_csv('flares_dataset.csv', index = False, columns = keywords_flare)
# ef.to_csv('ef_dataset.csv', index = False, columns = keywords_ef)
# cme.to_csv('cme_dataset.csv', index = False, columns = keywords_cme)
# ar.to_csv('ar_dataset.csv', index = False, columns = keywords_ar)
# sg.to_csv('sg_dataset.csv', index = False, columns = keywords_sigmoid)
# fe.to_csv('fe_dataset.csv', index = False, columns = keywords_filament_eruption)


In [5]:
client = hek.HEKClient()
start_time = pd.to_datetime('2014/01/01 00:00')
end_time = pd.to_datetime('2014/02/01 00:00')

ar_events1 = client.query(hek.attrs.Time(start_time.isoformat(),
                                                 end_time.isoformat()),
                                  hek.attrs.EventType('AR'))

ar_data1 = pd.DataFrame(ar_events1)
print ar_data1

                        SOL_standard  absnetcurrenthelicity active  \
0     SOL2013-12-31T20:00:00L223C106             162.794006   true   
1     SOL2013-12-31T20:00:00L154C103               7.397000   true   
2     SOL2013-12-31T20:00:00L125C081              52.464001   true   
3     SOL2013-12-31T20:00:00L268C107              71.862999   true   
4     SOL2013-12-31T20:00:00L144C112               8.802000   true   
5     SOL2013-12-31T20:00:00L175C072               3.010000   true   
6     SOL2013-12-31T20:00:00L185C097               0.452000   true   
7     SOL2013-12-31T20:00:00L179C105              83.500999   true   
8     SOL2013-12-31T20:20:36L227C106                    NaN   true   
9     SOL2013-12-31T20:20:36L156C103                    NaN   true   
10    SOL2013-12-31T20:20:36L116C103                    NaN   true   
11    SOL2013-12-31T20:20:36L122C080                    NaN   true   
12    SOL2013-12-31T20:20:36L121C119                    NaN   true   
13    SOL2013-12-31T

In [28]:
def collect_ar_noaa(df):
    unique_noaa = df.ar_noaanum.unique()
    keywords_ar = list(np.genfromtxt('keywords_ar.csv', delimiter=',', dtype =str))
    noaa_dic = {elem : pd.DataFrame for elem in unique_noaa}
    start_times = []
    end_times= []
    file_names = []
    i = 0
    for key in noaa_dic.keys():
        if i != 0:
            noaa_dic[key] = df[:][df.ar_noaanum == key]
            start_time = noaa_dic[key]['event_starttime'].values[0]
            end_time = noaa_dic[key]['event_endtime'].values[-1]
            start_times.append(start_time)
            end_times.append(end_time)
            fName = str(key)[:-2] +'_'+ start_time+'_'+end_time
            location = 'ar_w_noaa_num/' + fName+ '.csv'
            noaa_dic[key].to_csv(location, columns = keywords_ar, index = False)
            file_names.append(fName)
        i+=1  
    file_names = np.array(file_names).T
    with open('ar_w_noaa_num/file_names.csv','w') as f:
        np.savetxt(f, X =file_names, fmt = '%s', delimiter=',')
 
collect_ar_noaa(ar_data1)

#, hek.attrs.AR.NOAANum != None

In [ ]:
def merge_ar_sources(inputFile):
    ar_set =pd.read_csv(inputFile, delimiter = ',', header = 0)
    ar_set_NOAA = ar_set.ix[ar_set['frm_name']=='NOAA SWPC Observer']

    frm_name
    'HMI SHARP'
    'NOAA SWPC Observer'